# 문서 검색 효율화를 위한 기계독해
- 1차 모의경진대회(22.11.14 ~ 22.11.25)
- 자연어 기계독해(Machine Reading Comprehension) 과제

## 데이터 구조

```
$ MRC/
├── DATA/
│   ├── train.json
│   ├── test.json
│   └── sample_submission.csv
├── prediction.csv (코드 실행 후 생성)
├── results/ (코드 실행 후 생성)
```

In [2]:
!nvidia-smi

Mon Nov 21 23:19:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.72       Driver Version: 410.72       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   28C    P0    24W / 250W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# 0. 사전 준비

## 0.1 구글 드라이브 마운트

In [3]:
# # 구글 Colaboratory 를 사용하기 위해 구글 계정으로 로그인합니다. 
# from google.colab import drive
# drive.mount('/content/drive')

## 0.2 라이브러리 설치

In [4]:
!pip install transformers

## 1. 라이브러리 불러오기

In [5]:
import os
import sys
import csv
import copy
import json
import random
import shutil
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from datetime import datetime, timezone, timedelta

from transformers import ElectraTokenizerFast, ElectraTokenizer
from transformers import ElectraForQuestionAnswering

###
from transformers import ElectraModel
from tokenizers import Encoding
###


import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## 2. 하이퍼파라미터 및 기타 인자 설정

### 2.1 데이터 경로

In [6]:
PROJECT_DIR = './MRC'
DATA_DIR= './MRC/DATA'

### 2.2 시드 설정

In [7]:
# 난수 생성기가 항상 일정한 값을 출력하게 하기 위해 seed 고정
RANDOM_SEED = 42

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

### 2.3 하이퍼파라미터 설정

In [8]:
LEARNING_RATE = 1e-5     # 학습률(learning rate)은 경사하강법(gradient descent)을 통해 내리막길을 내려갈 때의 보폭
BATCH_SIZE = 4     # 배치(batch)는 모델의 가중치(weights)를 업데이트하는 학습 데이터의 단위. 여기서는 16개를 학습할 때마다 모델의 가중치(weights)를 업데이트한다는 것
PIN_MEMORY = True
NUM_WORKERS = 0
EPOCHS = 6     # 에폭은 전체 학습 데이터를 학습에 사용하는 횟수. 주어진 학습 데이터를 여러번 학습할 수 있음
DROP_LAST = False
EARLY_STOPPING_MODE = min
EARLY_STOPPING_PATIENCE = 10
EARLY_STOPPING_TARGET = 'val_loss'     # validation set의 loss를 기준으로 early_stopping 여부를 결정할 것
LOGGING_INTERVAL = 200

### 2.4 디바이스 설정

In [9]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 3. Dataset 정의

In [10]:
class QADataset(Dataset):     # 데이터를 input으로 변환해주는 Dataset 클래스를 상속하여, QA(Question Answering) 과제에 맞게 커스터마이징한다
    
    def __init__ (self, data_dir: str, tokenizer, max_seq_len: int, mode = 'train'):     # Dataset 클래스는 기본적으로 __init__, __len__, __getitem__를 정의해 주어야 한다
        self.mode = mode
        self.data = json.load(open(data_dir, 'r', encoding='utf8'))
        
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        
        if mode == 'test':
            self.encodings, self.question_ids = self.preprocess()
        else:
            self.encodings, self.answers = self.preprocess()
        
    def __len__(self):     # index를 통해 input을 순차적으로 읽어오기 위해서는 데이터의 길이가 먼저 확인되어야 한다. __len__ 함수는 input의 길이를 반환해주는 함수
        return len(self.encodings.input_ids)

    def __getitem__(self, index: int):     # input의 길이가 확인되면 index를 통해 데이터를 불러올 수 있다. __getitem__ 함수는 index에 해당하는 input 데이터를 반환해주는 함수
        return {key: torch.tensor(val[index]) for key, val in self.encodings.items()}

    
    def preprocess(self):
        contexts, questions, answers, question_ids = self.read_squad()     # SQuAD(Stanford Question Answering Dataset) 형식의 데이터에서 contexts, questions, answers, question_ids를 읽어오는 함수
        if self.mode == 'test':
            encodings = self.tokenizer(contexts, questions, truncation=True, max_length = self.max_seq_len, padding=True)
            return encodings, question_ids
        else: # train or val
            self.add_end_idx(answers, contexts)     # train.json에는 질문에 대한 답이 context 내에서 시작되는 index인 'answer_srart'만 있기 때문에, 추가로 'answer_end'를 찾아주는 함수
            encodings = self.tokenizer(contexts, questions, truncation=True, max_length = self.max_seq_len, padding=True)
            self.add_token_positions(encodings, answers)
        
            return encodings, answers
        
    
    def read_squad(self):     # SQuAD(Stanford Question Answering Dataset) 형식의 데이터에서 contexts, questions, answers, question_ids를 읽어오는 함수
        contexts = []
        questions = []
        question_ids = []
        answers = []
        
        # train - val split
        if self.mode == 'train':
            self.data['data'] = self.data['data'][:-1*int(len(self.data['data'])*0.2)]
        elif self.mode == 'val':
            self.data['data'] = self.data['data'][-1*int(len(self.data['data'])*0.2):]
        
        
        till = len(self.data['data'])
        

        for group in self.data['data'][:till]:
            for passage in group['paragraphs']:
                context = passage['context']
                for qa in passage['qas']:
                    question = qa['question']
                    if self.mode == 'test':
                        contexts.append(context)
                        questions.append(question)
                        question_ids.append(qa['question_id'])
                    else: # train or val
                        for ans in qa['answers']:
                            contexts.append(context)
                            questions.append(question)

                            if qa['is_impossible']:
                                answers.append({'text':'','answer_start':-1})
                            else:
                                answers.append(ans)
                
        # return formatted data lists
        return contexts, questions, answers, question_ids
    
    
    def add_end_idx(self, answers, contexts):     # train.json에는 질문에 대한 답이 context 내에서 시작되는 index인 'answer_srart'만 있기 때문에, 추가로 'answer_end'를 찾아주는 함수
        for answer, context in zip(answers, contexts):
            gold_text = answer['text']
            start_idx = answer['answer_start']
            end_idx = start_idx + len(gold_text)

            # in case the indices are off 1-2 idxs
            if context[start_idx:end_idx] == gold_text:
                answer['answer_end'] = end_idx
            else:
                for n in [1, 2]:
                    if context[start_idx-n:end_idx-n] == gold_text:
                        answer['answer_start'] = start_idx - n
                        answer['answer_end'] = end_idx - n
                    elif context[start_idx+n:end_idx+n] == gold_text:
                        answer['answer_start'] = start_idx + n
                        answer['answer_end'] = end_idx + n
                        

    def add_token_positions(self, encodings, answers):
        # should use Fast tokenizer
        start_positions = []
        end_positions = []
        for i in range(len(answers)):
            if answers[i]['answer_start'] == -1:
                # set [CLS] token as answer if is_impossible
                start_positions.append(0)
                end_positions.append(1)
            else:
                start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))

                assert 'answer_end' in answers[i].keys(), f'no answer_end at {i}'
                end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

            # answer passage truncated
            if start_positions[-1] is None:
                start_positions[-1] = tokenizer.model_max_length                
            # end position cannot be found, shift until found
            shift = 1
            while end_positions[-1] is None:
                end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
                shift += 1
                
        # char-based -> token based
        encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

## 4. 모델 정의

In [11]:
class electra(nn.Module):     # pytorch의 모든 neural network 모델들은 torch.nn.Module 클래스를 상속해야 한다. 기본적으로 __init__()과 forward 함수가 override(재정의)되어야 하며, forward 함수는 모델의 계산을 실행하는 것을 뜻한다.

    def __init__(self, pretrained, **kwargs):
        super(electra, self).__init__()

        self.model = ElectraForQuestionAnswering.from_pretrained(pretrained)     # Hugging Face에서 pretrain된 모델을 가져와서 model 변수에 저장한다.


    def forward(self, input_ids, attention_mask, start_positions=None, end_positions=None):
        
        outputs = self.model(input_ids = input_ids, 
                             attention_mask = attention_mask,
                             start_positions = start_positions,
                             end_positions = end_positions)
        
        return outputs

## 5. Utils 정의
### 5.1 EarlyStopper

In [12]:
class EarlyStopper():     # 일정 기간 모델 성능에 개선이 없으면, 학습을 중단하는 기능

    def __init__(self, patience: int, mode:str)-> None:
        self.patience = patience
        self.mode = mode

        # Initiate
        self.patience_counter = 0
        self.stop = False
        self.best_loss = np.inf

        print(f"Initiated early stopper, mode: {self.mode}, best score: {self.best_loss}, patience: {self.patience}")

        
    def check_early_stopping(self, loss: float)-> None:
        loss = -loss if self.mode == 'max' else loss  # get max value if mode set to max

        if loss > self.best_loss:
            # got worse score
            self.patience_counter += 1

            print(f"Early stopper, counter {self.patience_counter}/{self.patience}, best:{abs(self.best_loss)} -> now:{abs(loss)}")
            
            if self.patience_counter == self.patience:
                print(f"Early stopper, stop")
                self.stop = True  # end

        elif loss <= self.best_loss:
            # got better score
            self.patience_counter = 0
            
            print(f"Early stopper, counter {self.patience_counter}/{self.patience}, best:{abs(self.best_loss)} -> now:{abs(loss)}")
            print(f"Set counter as {self.patience_counter}")
            print(f"Update best score as {abs(loss)}")
            
            self.best_loss = loss
            
        else:
            print('debug')

### 5.2 Trainer

In [13]:
class Trainer():     # 학습을 위한 Trainer 클래스 정의

    def __init__(self,
                 model,
                 optimizer,
                 loss,
                 metrics,
                 device,
                 tokenizer,
                 interval=100):
        
        self.model = model
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
        self.device = device
        self.interval = interval
        self.tokenizer = tokenizer

        # History
        self.loss_sum = 0  # Epoch loss sum
        self.loss_mean = 0 # Epoch loss mean
        self.y = list()
        self.y_preds = list()
        self.score_dict = dict()  # metric score
        self.elapsed_time = 0
        

    def train(self, mode, dataloader, tokenizer, epoch_index=0):
        
        start_timestamp = time()
        self.model.train() if mode == 'train' else self.model.eval()     # 모델을 train(eval) mode로 전환.  train(eval) mode에서는 dropout, batchnorm이 적용된다(적용되지 않는다)
 
        for batch_index, batch in enumerate(tqdm(dataloader, leave=True)):
            
            self.optimizer.zero_grad()     # 파라미터 업데이트는 batch 단위로 이루어지고, 매 batch마다 이전 스템에서 계산된 gradient를 초기화해주어야 함
            # pull all the tensor batches required for training
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            start_positions = batch['start_positions'].to(self.device)
            end_positions = batch['end_positions'].to(self.device)
            
            # train model on batch and return outputs (incl. loss)
            # Inference
            outputs = self.model(input_ids, attention_mask=attention_mask,
                            start_positions=start_positions,
                            end_positions=end_positions)
            
            loss = outputs.loss
            start_score = outputs.start_logits
            end_score = outputs.end_logits
            
            
            start_idx = torch.argmax(start_score, dim=1).cpu().tolist()
            end_idx = torch.argmax(end_score, dim=1).cpu().tolist()
            
            # Update
            if mode == 'train':
                loss.backward()     # backpropagation
                self.optimizer.step()     # 파라미터 업데이트
                
            elif mode in ['val', 'test']:
                pass
            
            # History
            self.loss_sum += loss.item()
            
            # create answer; list of strings
            for i in range(len(input_ids)):
                if start_idx[i] > end_idx[i]:
                    output = ''
                
                self.y_preds.append(self.tokenizer.decode(input_ids[i][start_idx[i]:end_idx[i]]))
                self.y.append(self.tokenizer.decode(input_ids[i][start_positions[i]:end_positions[i]]))


            # Logging
            if batch_index % self.interval == 0:
                print(f"batch: {batch_index}/{len(dataloader)} loss: {loss.item()}")
                
        # Epoch history
        self.loss_mean = self.loss_sum / len(dataloader)  # Epoch loss mean

        # Metric
        score = self.metrics(self.y, self.y_preds)
        self.score_dict['metric_name'] = score

        # Elapsed time
        end_timestamp = time()
        self.elapsed_time = end_timestamp - start_timestamp

    def clear_history(self):
        self.loss_sum = 0
        self.loss_mean = 0
        self.y_preds = list()
        self.y = list()
        self.score_dict = dict()
        self.elapsed_time = 0

### 5.3 Recorder

In [14]:
class Recorder():

    def __init__(self,
                 record_dir: str,
                 model: object,
                 optimizer: object):
        
        self.record_dir = record_dir
        self.record_filepath = os.path.join(self.record_dir, 'record.csv')
        self.weight_path = os.path.join(record_dir, 'model.pt')

        self.model = model
        self.optimizer = optimizer

        
    def set_model(self, model: 'model'):
        self.model = model


    def add_row(self, row_dict: dict):

        fieldnames = list(row_dict.keys())

        with open(self.record_filepath, newline='', mode='a') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)

            if f.tell() == 0:
                writer.writeheader()

            writer.writerow(row_dict)
            print(f"Write row {row_dict['epoch_index']}")

            
    def save_weight(self, epoch: int)-> None:
        check_point = {
            'epoch': epoch + 1,
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
        }
        
        torch.save(check_point, self.weight_path)
        print(f"Recorder, epoch {epoch} Model saved: {self.weight_path}")

## 6. 모델 학습

### 6.1 모델과 기타 utils 설정

In [ ]:
# Load model
model = electra(pretrained="monologg/koelectra-base-v3-discriminator").to(device)

# Set optimizer, loss function, metric function
optimizer = optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
loss = F.cross_entropy
metrics = accuracy_score

# Set tokenizer
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")

# Set Trainer
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  loss=loss,
                  metrics=metrics,
                  device=device,
                  tokenizer=tokenizer,
                  interval=LOGGING_INTERVAL)

# Set earlystopper
early_stopper = EarlyStopper(patience=EARLY_STOPPING_PATIENCE,
                            mode=min)

# Set train serial
kst = timezone(timedelta(hours=9))
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")


# Set recorder 
RECORDER_DIR = os.path.join(PROJECT_DIR, 'results', 'train', train_serial)
os.makedirs(RECORDER_DIR, exist_ok=True)

recorder = Recorder(record_dir=RECORDER_DIR,
                    model=model,
                    optimizer=optimizer)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.bias', 

Initiated early stopper, mode: <built-in function min>, best score: inf, patience: 10


### 6.2 Dataset & Dataloader 설정

In [ ]:
# torch.utils.data.Dataset : 데이터를 input으로 변환
train_dataset = QADataset(data_dir=os.path.join(DATA_DIR, 'train.json'), tokenizer = tokenizer, max_seq_len = 512, mode = 'train')
val_dataset = QADataset(data_dir=os.path.join(DATA_DIR, 'train.json'), tokenizer = tokenizer, max_seq_len = 512, mode = 'val')

# torch.utils.data.DataLoader : input을 배치 단위로 리턴해주는 기능
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS, 
                              shuffle=True,
                              pin_memory=PIN_MEMORY,
                              drop_last=DROP_LAST)

val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=BATCH_SIZE,
                            num_workers=NUM_WORKERS, 
                            shuffle=False,
                            pin_memory=PIN_MEMORY,
                            drop_last=DROP_LAST)

print(f"Load data, train:{len(train_dataset)} val:{len(val_dataset)}")

Load data, train:22387 val:5637


### 6.3 Epoch 단위 학습 진행

In [17]:
# Train
for epoch_index in range(EPOCHS):

    # Set Recorder row
    row_dict = dict()
    row_dict['epoch_index'] = epoch_index
    row_dict['train_serial'] = train_serial

    """
    Train
    """
    print(f"Train {epoch_index}/{EPOCHS}")
    print(f"--Train {epoch_index}/{EPOCHS}")
    trainer.train(dataloader=train_dataloader, epoch_index=epoch_index, tokenizer=tokenizer, mode='train')

    row_dict['train_loss'] = trainer.loss_mean
    row_dict['train_elapsed_time'] = trainer.elapsed_time 

    for metric_str, score in trainer.score_dict.items():
        row_dict[f"train_{metric_str}"] = score
    trainer.clear_history()

    """
    Validation
    """
    print(f"Val {epoch_index}/{EPOCHS}")
    print(f"--Val {epoch_index}/{EPOCHS}")
    trainer.train(dataloader=val_dataloader, epoch_index=epoch_index, tokenizer=tokenizer, mode='val')

    row_dict['val_loss'] = trainer.loss_mean
    row_dict['val_elapsed_time'] = trainer.elapsed_time 

    for metric_str, score in trainer.score_dict.items():
        row_dict[f"val_{metric_str}"] = score
    trainer.clear_history()

    """
    Record
    """
    recorder.add_row(row_dict)

    """
    Early stopper
    """
    early_stopping_target = EARLY_STOPPING_TARGET
    early_stopper.check_early_stopping(loss=row_dict[early_stopping_target])

    if early_stopper.patience_counter == 0:
        recorder.save_weight(epoch=epoch_index)
        best_row_dict = copy.deepcopy(row_dict)

    if early_stopper.stop == True:
        print(f"Early stopped, counter {early_stopper.patience_counter}/{EARLY_STOPPING_PATIENCE}")

        break

Train 0/6
--Train 0/6


  0%|          | 2/5597 [00:00<24:15,  3.85it/s]

batch: 0/5597 loss: 6.099382400512695


  4%|▎         | 202/5597 [00:27<12:07,  7.41it/s]

batch: 200/5597 loss: 2.3963546752929688


...

batch: 1200/1410 loss: 2.6980364322662354


100%|█████████▉| 1404/1410 [01:01<00:00, 22.77it/s]

batch: 1400/1410 loss: 2.6678555011749268


100%|██████████| 1410/1410 [01:01<00:00, 22.77it/s]

Write row 5
Early stopper, counter 4/10, best:0.5513570183278755 -> now:0.9161958823309102


## 7. 추론

### 7.1 테스트 Dataset & Dataloader 설정

In [18]:
# Load data
test_dataset = QADataset(data_dir=os.path.join(DATA_DIR, 'test.json'), tokenizer = tokenizer, max_seq_len = 512, mode = 'test')

question_ids = test_dataset.question_ids

test_dataloader = DataLoader(dataset=test_dataset,
                            batch_size=BATCH_SIZE,
                            num_workers=NUM_WORKERS, 
                            shuffle=False,
                            pin_memory=PIN_MEMORY,
                            drop_last=DROP_LAST)

### 7.2 모델 로드

In [19]:
# Load model

model = electra(pretrained="monologg/koelectra-base-v3-discriminator").to(device)

checkpoint = torch.load(os.path.join(RECORDER_DIR, 'model.pt'))

model.load_state_dict(checkpoint['model'])

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.bias', 

<All keys matched successfully>

### 7.3 추론 진행

In [20]:
model.eval()     # 모델을 eval mode로 전환. train mode와 달리 eval mode에서는 dropout, batchnorm이 적용되지 않는다

pred_df = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))

for batch_index, batch in enumerate(tqdm(test_dataloader, leave=True)):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    # Inference
    outputs = model(input_ids, attention_mask=attention_mask)

    start_score = outputs.start_logits
    end_score = outputs.end_logits

    start_idx = torch.argmax(start_score, dim=1).cpu().tolist()
    end_idx = torch.argmax(end_score, dim=1).cpu().tolist()

    y_pred = []
    for i in range(len(input_ids)):
        if start_idx[i] > end_idx[i]:
            output = ''

        ans_txt = tokenizer.decode(input_ids[i][start_idx[i]:end_idx[i]]).replace('#','')

        if ans_txt == '[CLS]':
            ans_txt == ''

        y_pred.append(ans_txt)


    q_end_idx = BATCH_SIZE*batch_index + len(y_pred)
    for q_id, pred in zip(question_ids[BATCH_SIZE*batch_index:q_end_idx], y_pred):
        pred_df.loc[pred_df['question_id'] == q_id,'answer_text'] = pred

100%|██████████| 407/407 [00:14<00:00, 27.90it/s]


### 7.4 결과 저장

In [21]:
# Set predict serial
kst = timezone(timedelta(hours=9))
predict_timestamp = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")
predict_serial = predict_timestamp
predict_serial

PREDICT_DIR = os.path.join(PROJECT_DIR, 'results', 'predict', predict_serial)
os.makedirs(PREDICT_DIR, exist_ok=True)

pred_df.to_csv(os.path.join(PREDICT_DIR, f'prediction_lr{LEARNING_RATE}_bs{BATCH_SIZE}_epoch{EPOCHS}_split0.2_koelectra-base.csv'), index=False)